In [1]:
import pandas as pd
from random import seed, sample

from IPython.display import display

from ipymarkup import show_markup, AsciiMarkup

from yargy import (
    Parser,
    or_, rule
)
from yargy.pipelines import morph_pipeline
from yargy.predicates import (
    eq, lte, gte, gram, type, tag,
    length_eq,
    in_, in_caseless, dictionary,
    normalized, caseless,
    is_title, custom
)

from yargy.tokenizer import MorphTokenizer
from yargy import interpretation as interp
from yargy.interpretation import fact, attribute

import sys
sys.path.append('/home/bun/natasha')

from natasha_expanded.grammars import address

In [6]:
data = pd.read_csv('scrp_data_lost_and_found_22_11_2018.csv')

lines = data['msg_text'].tolist()

In [159]:
def load_lines(path):
    with open(path) as file:
        for line in file:
            yield line.rstrip('\n')
            
            
METRO_STATIONS = list(load_lines('metro.txt'))

def load_text(path):
    with open(path) as file:
        return file.read()
    
metro_names = load_text('metro_names.csv').split('\n')

In [36]:
def show_matches(rule, *lines):
    parser = Parser(rule)
    for line in lines:
        matches = parser.findall(line)
        matches = sorted(matches, key=lambda _: _.span)
        spans = [_.span for _ in matches]
        show_markup(line, spans)

In [2]:
import requests
from functools import lru_cache

YNDX_GEO_KEY = '29d541ba-6887-4e71-be53-b00d7e178147'

@lru_cache(maxsize=2**11)
def get_coordinates(geocode):
    r = requests.get(f'https://geocode-maps.yandex.ru/1.x/?apikey={YNDX_GEO_KEY}&geocode={geocode}&format=json&results=1')
    featureMember = r.json()['response']['GeoObjectCollection']['featureMember']
    if featureMember:
        r = featureMember[0]['GeoObject']['Point']['pos']
        long, lat = [float(i) for i in r.split(' ')]
        return long, lat
    else:
        return None, None

In [2]:
FIND_ADDRESS = [
    address.METRO,
    address.STREET_LEVEL,
    address.GOROD_LEVEL,
    address.RAION]

def find_address(text):
    for lev in FIND_ADDRESS:
        pars = Parser(lev)
        matc = pars.findall(text)
        facts = [_.span for _ in matc]
        if facts:
            for fact in facts:
                return text[fact[0]:fact[1]], get_coordinates(text[fact[0]:fact[1]])

In [11]:
Address = fact(
    'Address',
    [attribute('parts').repeatable()]
)

In [191]:
FIND_ADDRESS = or_(
    address.METRO.interpretation(
        Address.parts
    ),
    address.STREET_LEVEL_CUSTOM.interpretation(
        Address.parts
    ),
    address.GOROD_LEVEL.interpretation(
        Address.parts
    ),
    address.RAION.interpretation(
        Address.parts
    ),
    OTHER_OBJECTS.interpretation(
        Address.parts
    )
).interpretation(Address)

In [218]:
def find_address(text):
        pars = Parser(FIND_ADDRESS)
        matc = pars.findall(text)
        #print(list(matc))        
        facts = [_.span for _ in matc]
        return ' '.join([text[f[0]:f[1]] for f in facts])

In [219]:
text = """5.12.17 №32214 кобель, Взрослый Черный с белым,хвост пушистый,не купированный, больше темного окраса,или просто грязный. в районе Можайское шоссе, бегает собака(мальчик),грязный,похож на русского спаниеля. Заметили его дней 5-6 тому назад,близко не подходит,кажется с коричневым ошейником,упитанный.может кто ищет Найден(а) в районе Москва ул.Толбухина, ул Говорово 89163894451 Ирина mussirina@mail.ru"""

In [216]:
pars = Parser(FIND_ADDRESS)
matc = pars.findall(text)
facts = [_.fact for _ in matc]
print(facts[0].parts)

[Street(name='Можайское', type='шоссе')]


In [221]:
get_coordinates(find_address(text))

(37.395509, 55.718796)

In [116]:
show_matches(FIND_ADDRESS, text)

BoxMarkup('5.12.17 №32214 кобель, Взрослый Черный с белым,хвост пушистый,не купированный, больше темного окраса,или просто грязный. в районе Можайское шоссе, бегает собака(мальчик),грязный,похож на русского спаниеля. Заметили его дней 5-6 тому назад,близко не подходит,кажется с коричневым ошейником,упитанный.может кто ищет Найден(а) в районе Москва ул.Толбухина, ул Говорово 89163894451 Ирина mussirina@mail.ru',
          [Span(130, 146, None),
           Span(333, 339, None),
           Span(340, 353, None),
           Span(354, 365, None)])

In [125]:
get_coordinates('Москва Метро Щукинская улице Академика Бочвара')

(37.458732, 55.804172)

In [217]:
for line in sample(lines, 10):
    show_matches(FIND_ADDRESS, line)
    print(find_address(line))

BoxMarkup('Сегодня, 13.05.2018 №33091 кобель, Около года Совсем молодой курцхар — белый с коричневым крапом, тёмно- коричневые уши и большое ровное круглое пятно на крупе. Доброжелательный, но в руки не даётся. Тянется к собакам. 2 дорогих ошейника. Адресника и клейм в ушах при кормежке не заметили. Найден(а) в районе Московская область, Дмитровский район, село Орудьево, СНТ Экспресс 9057375317 Елена s_elena2001@mail.ru',
          [Span(309, 319, None), Span(329, 346, None), Span(348, 361, None)])

Московская Дмитровский район село Орудьево


BoxMarkup('Долгопрудный, Проспект Пацаева - найдена собака в ошейнике. Добрая, команды выполняет, игривая. По дороге бродила. Чуть машина не сбила. Хозяин, отзовись!',
          [Span(0, 12, None), Span(14, 30, None)])

Долгопрудный Проспект Пацаева


BoxMarkup('29.12.17 №32336 кобель,  Собачка не боится людей , слепая , с чёрным потертым ошейником Найден(а) в районе Можайское шоссе 89850693464 Валерия lerapups007@rambler.ru',
          [Span(107, 122, None)])

...

Угрешской ул. Конечная трамваев 43


BoxMarkup('22.11.2017 12:30 №32080 кобель, Около года В районе Открытого шоссе появился метис овчарки, молодой кобелёк (подросток), дружелюбный, послушный, очень любит детей. В каждом взрослом видит хозяина- ластится. Найден(а) в районе открытое шоссе 8-926-076-77-17 Ольга',
          [Span(52, 67, None), Span(226, 242, None)])

Открытого шоссе открытое шоссе 8


BoxMarkup('02.07.2017 №31062 сука, 1 год Молодая девочка белого цвета Найден(а) в районе Балашиха Московской области 89031551338 Елена e_lenka09@mail.ru',
          [Span(78, 86, None), Span(87, 97, None)])

Балашиха Московской


In [ ]:
около Бц Двинцев
 у ТЦ "Сота
станция ПРАВДА
районе СНТ "Сосновый бор" 
СНТ надежда
парка Фили
Садоводство Ларионово
на киевском вокзале
ПРОСПЕКТ КОМСОМОЛЬСКИЙ
сао
ст. Ильинская-быково